# Hometask

In [0]:
import time
import torch
import zipfile
import collections
from torch import nn
import math

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file

In [3]:
!git clone https://github.com/Phylliida/Dialogue-Datasets.git

Cloning into 'Dialogue-Datasets'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Unpacking objects: 100% (36/36), done.


In [4]:
with open('/content/Dialogue-Datasets/TwitterConvCorpus.txt') as f:
    raw_text = f.read()
print(raw_text[0:1000])

what's up dadyo when did you get back on Twitter? Haha
like 2 weeks ago and it's going as terribly as I remember, but Deg is still hilarious so it's ok
literally never about that account, love it.



Answer me this fellow Apple peoples: how many times in the past year have you used the escape key?
about 50 times today. Terminal vim user.
Seems the major complaints so far are from VIM users like yourself. I’m wondering how force quit is gonna work.
cmd+opt+esc is good but still available via menubar



There was a greasy kid at highline who was basically miles dipped in a bucket of ranch
I'm disgusted
he flashed us then we scored so he sadly put his shirt back on 😂#fuckhighline
what a piece of shit



I killed a mouse today!  😼
Yay, you great hunter. I’ve killed lots of lizards and bugs but never a mouse.
I don't know what the mouse was thinking??  Why come into a house with 2 ferocious felines??
and then that mouse had the nerve to try to eat our kibble!  Let this be a lesson fur all t

In [5]:
num_examples = 5000
source, target = [], []
for i, line in enumerate(raw_text.split('\n\n\n')):
    if i > num_examples:
        break
    if len(line.strip().split('\n')) <= 1:
      continue
    #print(line, line.strip().split('\n'))
    source.append(line.strip().split('\n')[0].split(' '))
    target.append(line.strip().split('\n')[1].split(' '))

source[0:3], target[0:3], len(source), len(target)

([["what's",
   'up',
   'dadyo',
   'when',
   'did',
   'you',
   'get',
   'back',
   'on',
   'Twitter?',
   'Haha'],
  ['Answer',
   'me',
   'this',
   'fellow',
   'Apple',
   'peoples:',
   'how',
   'many',
   'times',
   'in',
   'the',
   'past',
   'year',
   'have',
   'you',
   'used',
   'the',
   'escape',
   'key?'],
  ['There',
   'was',
   'a',
   'greasy',
   'kid',
   'at',
   'highline',
   'who',
   'was',
   'basically',
   'miles',
   'dipped',
   'in',
   'a',
   'bucket',
   'of',
   'ranch']],
 [['like',
   '2',
   'weeks',
   'ago',
   'and',
   "it's",
   'going',
   'as',
   'terribly',
   'as',
   'I',
   'remember,',
   'but',
   'Deg',
   'is',
   'still',
   'hilarious',
   'so',
   "it's",
   'ok'],
  ['about', '50', 'times', 'today.', 'Terminal', 'vim', 'user.'],
  ["I'm", 'disgusted']],
 1900,
 1900)

In [6]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = collections.Counter(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[0])
        self.token_freqs.sort(key=lambda x: x[1], reverse=True)
        if use_special_tokens:
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            uniq_tokens = ['<pad>', '<bos>', '<eos>', '<unk>']
        else:
            self.unk, uniq_tokens = 0, ['<unk>']
        uniq_tokens +=  [token for token, freq in self.token_freqs 
                         if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1
            
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def build_vocab(tokens):
    tokens = [token for line in tokens for token in line]
    return Vocab(tokens, min_freq=3, use_special_tokens=True)

src_vocab = build_vocab(source)
print(len(src_vocab))

def pad(line, max_len, padding_token):
    if len(line) > max_len:
        return line[:max_len]
    return line + [padding_token] * (max_len - len(line))

print(pad(src_vocab[source[0]], 10, src_vocab.pad))

1342
[335, 28, 3, 55, 136, 12, 31, 75, 16, 3]


In [0]:
def build_array(lines, vocab, max_len, is_source, build_mask=False):
    lines = [vocab[line] for line in lines]
    if not is_source:
        lines = [[vocab.bos] + line + [vocab.eos] for line in lines]
    array = torch.LongTensor([pad(line, max_len, vocab.pad) for line in lines])
    valid_len = (array != vocab.pad).sum(axis=1)
    
    if build_mask:
        mask = torch.zeros_like(array).type(torch.float)
        for i in range(len(valid_len)):
            mask[i, :valid_len[i]]=1.
        return array, valid_len, mask

    return array, valid_len

def load_data_nmt(batch_size, max_len): 
    src_vocab, tgt_vocab = build_vocab(source), build_vocab(target)
    src_array, src_valid_len = build_array(source, src_vocab, max_len, True, False)
    tgt_array, tgt_valid_len, tgt_mask = build_array(target, tgt_vocab, max_len, False, True)
    train_data = torch.utils.data.TensorDataset(
        src_array, src_valid_len, tgt_array, tgt_valid_len, tgt_mask)
    train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
    return src_vocab, tgt_vocab, train_iter

In [8]:
src_vocab, tgt_vocab, train_iter = load_data_nmt(batch_size=2, max_len=200)
for X, X_valid_len, Y, Y_valid_len, y_mask in train_iter:
    print('X =', X.type(torch.int), '\nValid lengths for X =', X_valid_len,
          '\nY =', Y.type(torch.int), '\nValid lengths for Y =', Y_valid_len,
          '\nY mask =', y_mask)
    break

X = tensor([[ 335,   28,    3,   55,  136,   12,   31,   75,   16,    3,    3,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    

In [0]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, num_hiddens, num_layers, dropout=dropout)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers

    def forward(self, X, *args):
        X = self.embedding(X) 
        X = X.permute(1, 0, 2)
        state = (torch.zeros((self.num_layers, X.shape[1],self.num_hiddens)), torch.zeros((self.num_layers, X.shape[1],self.num_hiddens)))
        out, state = self.rnn(X, state)
        return out, state

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)
        self.num_hiddens = num_hiddens
        self.embed_size = embed_size
        self.vocab_size = vocab_size

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        X = self.embedding(X).permute(1, 0, 2)
        out, state = self.rnn(X, state)
        out = self.dense(out.view(-1, self.num_hiddens)).view(out.shape[0], out.shape[1], self.vocab_size).permute(1, 0, 2)
        return out, state

class MLPAttention(nn.Module):  
    def __init__(self, inputs, units, dropout, **kwargs):
        super(MLPAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(inputs, units, bias=False)
        self.W_q = nn.Linear(inputs, units, bias=False)
        self.v = nn.Linear(units, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value):
        query, key = torch.tanh(self.W_k(query)), torch.tanh(self.W_q(key))
        # expand query to (batch_size, #querys, 1, units), and key to 
        # (batch_size, 1, #kv_pairs, units). Then plus them with broadcast.  
        features = query.unsqueeze(2) + key.unsqueeze(1)
        scores = self.v(features).squeeze(dim=-1)
        attention_weights = self.dropout(scores.softmax(dim=-1))
        return torch.bmm(attention_weights, value)

class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.attention_cell = MLPAttention(num_hiddens, num_hiddens, dropout)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size + num_hiddens, num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)
        self.num_hiddens = num_hiddens
        self.embed_size = embed_size
        self.vocab_size = vocab_size

    def init_state(self, enc_outputs, *args):
        return (enc_outputs[0].permute(1,0,2), enc_outputs[1])

    def forward(self, X, state):
        enc_outputs, hidden_state = state
        X = self.embedding(X).permute(1, 0, 2)
        outputs = []
        for x in X:
            # query shape: (batch_size, 1, hidden_size)
            query = hidden_state[0][-1].unsqueeze(1)
            # context has same shape as query
            context = self.attention_cell(query, enc_outputs, enc_outputs)
            # concatenate on the feature dimension
            x = torch.cat([context, x.unsqueeze(1)], dim=-1)
            # reshape x to (1, batch_size, embed_size+hidden_size)
            out, hidden_state = self.rnn(x.permute(1, 0, 2), hidden_state)
            outputs.append(out)
        outputs = self.dense(torch.cat(outputs, dim=0))
        return outputs.permute(1, 0, 2), [enc_outputs, hidden_state]
            
            
       # out, state = self.rnn(X, state)
       # out = self.dense(out.view(-1, self.num_hiddens)).view(out.shape[0], out.shape[1], self.vocab_size).permute(1, 0, 2)
       # return out, state

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)


def train(model, data_iter, lr, num_epochs, out_vocab_size):
    trainer = torch.optim.Adam(model.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss(reduction='none')
    tic = time.time()
    for epoch in range(1, num_epochs+1):
        l_sum, num_tokens_sum = 0.0, 0.0
        for batch in data_iter:
            trainer.zero_grad()
            X, X_vlen, Y, Y_vlen, mask = [x for x in batch]
            Y_input, Y_label, Y_vlen, mask = Y[:,:-1], Y[:,1:], Y_vlen-1, mask[:,1:]
            Y_hat, _ = model(X, Y_input, X_vlen, Y_vlen)
            
            l = (loss(Y_hat.reshape(-1,out_vocab_size), Y_label.reshape(-1)) * mask.reshape(-1)).sum() 
            
            l.backward()
            num_tokens = Y_vlen.sum().item()
            trainer.step()
            l_sum += l.item()
            num_tokens_sum += num_tokens

        print("epoch %d, loss %.3f, time %.1f sec" % (
            epoch, l_sum/num_tokens_sum, time.time()-tic))
        tic = time.time()

In [13]:
embed_size, num_hiddens, num_layers, dropout = 32, 32, 3, 0.0
batch_size, max_len = 64, 30
lr, num_epochs = 0.01, 200

src_vocab, tgt_vocab, train_iter = load_data_nmt(batch_size, max_len)
encoder = Encoder(
    len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = AttentionDecoder(
    len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
model = EncoderDecoder(encoder, decoder)
train(model, train_iter, lr, num_epochs, len(tgt_vocab))

epoch 1, loss 5.231, time 4.5 sec
epoch 2, loss 4.714, time 4.6 sec
epoch 3, loss 4.704, time 4.4 sec
epoch 4, loss 4.707, time 4.6 sec
epoch 5, loss 4.696, time 4.4 sec
epoch 6, loss 4.687, time 4.4 sec
epoch 7, loss 4.674, time 4.6 sec
epoch 8, loss 4.661, time 4.5 sec
epoch 9, loss 4.643, time 4.5 sec
epoch 10, loss 4.619, time 4.4 sec
epoch 11, loss 4.589, time 4.5 sec
epoch 12, loss 4.557, time 4.4 sec
epoch 13, loss 4.530, time 4.5 sec
epoch 14, loss 4.502, time 4.5 sec
epoch 15, loss 4.475, time 4.6 sec
epoch 16, loss 4.450, time 4.5 sec
epoch 17, loss 4.429, time 4.6 sec
epoch 18, loss 4.397, time 4.5 sec
epoch 19, loss 4.360, time 4.4 sec
epoch 20, loss 4.320, time 4.4 sec
epoch 21, loss 4.275, time 4.5 sec
epoch 22, loss 4.229, time 4.6 sec
epoch 23, loss 4.180, time 4.4 sec
epoch 24, loss 4.124, time 4.5 sec
epoch 25, loss 4.055, time 4.6 sec
epoch 26, loss 4.000, time 4.6 sec
epoch 27, loss 3.941, time 4.6 sec
epoch 28, loss 3.882, time 4.5 sec
epoch 29, loss 3.823, time 4.

In [0]:
def translate(model, src_sentence, src_vocab, tgt_vocab, max_len):
    src_tokens = src_vocab[src_sentence.lower().split(' ')]
    src_len = len(src_tokens)
    if src_len < max_len:
        src_tokens += [src_vocab.pad] * (max_len - src_len)
    enc_X = torch.LongTensor(src_tokens)
    enc_valid_length = torch.LongTensor([src_len])
    enc_outputs = model.encoder(enc_X.unsqueeze(axis=0), enc_valid_length)
    dec_state = model.decoder.init_state(enc_outputs, enc_valid_length)
    dec_X = torch.LongTensor([tgt_vocab.bos]).unsqueeze(axis=0)
    predict_tokens = []
    for _ in range(max_len):
        Y, dec_state = model.decoder(dec_X, dec_state)
        dec_X = Y.argmax(axis=2)
        py = dec_X.squeeze(axis=0).type(torch.long).item()
        if py == tgt_vocab.eos:
            break
        predict_tokens.append(py)
    return ' '.join(tgt_vocab.to_tokens(predict_tokens))

In [15]:
'''what's up dadyo when did you get back on Twitter? Haha
like 2 weeks ago and it's going as terribly as I remember, but Deg is still hilarious so it's ok

Answer me this fellow Apple peoples: how many times in the past year have you used the escape key?
about 50 times today. Terminal vim user.

There was a greasy kid at highline who was basically miles dipped in a bucket of ranch
I'm disgusted

I killed a mouse today!  😼
Yay, you great hunter. I’ve killed lots of lizards and bugs but never a mouse.'''

for sentence in ['Hey you! What''s your name?', 'Whats UP?', "How are you?", 'Hello!', 'what''s up dadyo when did you get back on Twitter? Haha', 'I killed a mouse today!']:
    print(sentence + ' => ' + translate(
        model, sentence, src_vocab, tgt_vocab, max_len))

Hey you! Whats your name? => <unk> <unk> <unk> is a <unk>
Whats UP? => <unk> your <unk> <unk> That would know that I can like I <unk> ppl or I'd don't have a <unk> <unk> than your day on some <unk> to lose lol.
How are you? => I got you some <unk> <unk> I was one out to should if it happens to get the <unk> he's over a <unk> <unk> <unk>
Hello! => <unk> I think <unk> <unk> <unk> or <unk> so <unk> <unk> <unk>
whats up dadyo when did you get back on Twitter? Haha => if I was <unk> movie that &amp; <unk> <unk> They never been <unk> <unk>
I killed a mouse today! => I don't <unk> it
